In [3]:
import csv
import api

from tqdm import tqdm

In [4]:
queryId = '3778082360928566917'
filename = "chunks\\features"

In [5]:
jsonOriginal = api.getJson(queryId)


In [6]:
jsonIds = api.getIdList(jsonOriginal)


In [7]:
def getJsonValues(jsonIds):
    r = []
    for id in jsonIds:
        try:
            listOfJson = api.getJsonResultsDetailed(queryId,id)
            for json in listOfJson:
                name = json[0]['value']
                declaryngType = json[1]['value']
                methodDescriptor = json[2]['value']
                returnType = json[3]['value']
                numArgument = json[4]['value']
                hasConstants = json[5]['value']
                r.append((id,name, declaryngType, methodDescriptor, returnType, numArgument, hasConstants))
        except:
            raise Exception('Error in getJsonValues')
    return r 


In [11]:
chunks = [jsonIds[x:x+50] for x in range(0, len(jsonIds), 50)]

for i in tqdm(range(len(chunks))):
    idchunk = chunks[i]
    f = open(f'{filename}-chunk-{i}.csv', "w+")
    header = ['id','name', 'declaryngType', 'methodDescriptor', 'returnType', 'numArgument', 'hasConstants']	
    writer = csv.writer(f)
    writer.writerow(header)
    f.close()
    jsonFinal = getJsonValues(idchunk)


    with open(f'{filename}-chunk-{i}.csv', 'a', encoding='UTF8', newline='') as file:
        writer = csv.writer(file)
        for tuple in jsonFinal:
            id, name, dType, methodDescriptor, returnType, numArgument, hasConstants = tuple[0], tuple[1], tuple[2], tuple[3], tuple[4], tuple[5], tuple[6]
            row = [id, name, dType, methodDescriptor, returnType, numArgument, hasConstants]
            writer.writerow(row)
    i = 1 + i

100%|██████████| 20/20 [38:20<00:00, 115.02s/it] 


In [14]:
import os
import glob
import pandas as pd
os.chdir("\chunks")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

FileNotFoundError: [WinError 2] The system cannot find the file specified: '\\chunks'

In [22]:
import pandas as pd
chunkfiles = glob.glob("chunks\*.csv") 

In [23]:
df = pd.concat((pd.read_csv(f, header = 0) for f in chunkfiles))
df.to_csv("features.csv")